<a href="https://colab.research.google.com/github/bigear2021/geek-training-camp-homework/blob/main/target_mean_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

target_mean函数作业  
陈卓

In [ ]:
%load_ext Cython

In [55]:
#课堂代码
# distutils: language=c++
%%cython
import numpy as np
cimport numpy as np
import pandas as pd
import time


def hello():
    print("hello")


def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

cpdef target_mean_v3(data, y_name, x_name):
    cdef long nrow = data.shape[0]
    cdef np.ndarray[double] result = np.asfortranarray(np.zeros(nrow), dtype=np.float64)
    cdef np.ndarray[double] y = np.asfortranarray(data[y_name], dtype=np.float64)
    cdef np.ndarray[double] x = np.asfortranarray(data[x_name], dtype=np.float64)

    target_mean_v3_impl(result, y, x, nrow)
    return result

cdef void target_mean_v3_impl(double[:] result, double[:] y, double[:] x, const long nrow):
    cdef dict value_dict = dict()
    cdef dict count_dict = dict()

    cdef long i
    for i in range(nrow):
        if x[i] not in value_dict.keys():
            value_dict[x[i]] = y[i]
            count_dict[x[i]] = 1
        else:
            value_dict[x[i]] += y[i]
            count_dict[x[i]] += 1

    i=0
    for i in range(nrow):
        result[i] = (value_dict[x[i]] - y[i])/(count_dict[x[i]]-1)

In [56]:
def main():
    y = np.random.randint(2, size=(100000, 1))
    x = np.random.randint(10, size=(100000, 1))
    data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
    start=time.time()
    result = target_mean_v3(data, 'y', 'x')
    end=time.time()
    print(end-start)


if __name__ == '__main__':
    print('teacher time')
    main()

teacher time
0.05012845993041992


In [53]:
#本人代码
%%cython
import numpy as np
cimport numpy as np
import pandas as pd
import time

#修改点:都使用整型
cpdef target_mean_v3(data, y_name, x_name):
#修改点：避免负下标索引
    cdef unsigned int nrow = data.shape[0]
    cdef np.ndarray[long] result = np.asfortranarray(np.zeros(nrow), dtype=np.int64)
    cdef np.ndarray[long] y = np.asfortranarray(data[y_name], dtype=np.int64)
    cdef np.ndarray[long] x = np.asfortranarray(data[x_name], dtype=np.int64)

    target_mean_v3_impl(result, y, x, nrow)
    return result

cdef void target_mean_v3_impl(long[:] result, long[:] y, long[:] x, const long nrow):
    cdef dict value_dict = dict()
    cdef dict count_dict = dict()
#修改点：避免负下标索引
    cdef unsigned int i
    for i in range(nrow):
        if x[i] not in value_dict.keys():
            value_dict[x[i]] = y[i]
            count_dict[x[i]] = 1
        else:
            value_dict[x[i]] += y[i]
            count_dict[x[i]] += 1

    i=0
    for i in range(nrow):
        result[i] = (value_dict[x[i]] - y[i])/(count_dict[x[i]]-1)

In [54]:
def main():
    y = np.random.randint(2, size=(100000, 1))
    x = np.random.randint(10, size=(100000, 1))
    data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
    start=time.time()
    result = target_mean_v3(data, 'y', 'x')
    end=time.time()
    print(end-start)


if __name__ == '__main__':
    print('my time')
    main()

my time
0.02729177474975586
